### LA CRISE ECONOMIQUE EN BELGIQUE ENTRE 1929 ET 1933

###Il est question de réaliser une étude comparative de deux journaux belges: Le Drapeau Rouge et La Libre Belgique; 
afin de voir les secteurs mis en avant pendant cette période

### I- Création de nos corpus d'analyse


###A partir des fichiers extraits de la base de données CAMILLE, nous allons créer nos deux grands corpus d'analyse

In [28]:
## IMPORTS
import os
import textract

#Le dossier Data1 est constitué des fichiers relatifs aux articles publiés dans le journal Le Drapeau Rouge pendant ladite période (1929-1933)

In [90]:
txt_path1 = '../data/Data1_Le Drapeau Rouge'
if not os.path.exists(txt_path1):
    os.mkdir(txt_path1)

#Le dossier Data2 quant à lui, est constitué d'articles publiés dans le journal La Libre Belgique

In [52]:
txt_path2 = '../data/Data2_La libre Belgique'
if not os.path.exists(txt_path2):
    os.mkdir(txt_path2)

# pas important

In [31]:
txts = []
for f in os.listdir(txt_path1):
    if os.path.isfile(os.path.join(txt_path1, f)):
        txts.append(f)
txts[:5]

['KB_JB1051_1929-01-04_01-00002.txt',
 'KB_JB1051_1929-01-04_01-00004.txt',
 'KB_JB1051_1929-01-06_01-00002.txt',
 'KB_JB1051_1929-01-09_01-00003.txt',
 'KB_JB1051_1929-01-11_01-00003.txt']

In [44]:
txts = []
for f in os.listdir(txt_path2):
    if os.path.isfile(os.path.join(txt_path2, f)):
        txts.append(f)
txts[:5]

['KB_JB427_1929-01-26_01-00001.txt',
 'KB_JB427_1929-01-30_01-00003.txt',
 'KB_JB427_1929-04-12_01-00002.txt',
 'KB_JB427_1929-06-11_01-00003.txt',
 'KB_JB427_1929-11-06_01-00001.txt']

##Création du corpus du journal Drapeau Rouge

In [91]:
with open("../data/Data1_Le Drapeau Rouge.txt", "w", encoding="utf-8") as output_file:
    for file in os.listdir(txt_path1):
        if file.endswith(".txt"):
            with open(os.path.join(txt_path1, file), "r", encoding="utf-8") as f:
                output_file.write(f.read())


##Création du corpus du journal La Libre Belgique

In [71]:
with open("../data/Data2_La libre Belgique.txt", "w", encoding="utf-8") as output_file:
    for file in os.listdir(txt_path2):
        if file.endswith(".txt"):
            with open(os.path.join(txt_path2, file), "r", encoding="utf-8") as f:
                output_file.write(f.read())
                

#Nous venons ainsi de transformer les données que nous avons précedemment  extrait de la data base CAMILLE afin de constituer les corpus que nous allons ensuite exploiter tout au long de notre investigation.

### II-Constatation des déchets et nettoyage

##Dans cette partie, il est question de débarrasser nos corpus de leurs  déchets. Pour ce faire, nous allons commencer par constater les déchets dans nos deux corpus, ensuite nous allons créer une liste de stopwords et enfin nous allons créer une fonction de nettoyage pour chacun de nos corpus.

1-Importation des librairies et stopwords

In [55]:
from collections import Counter
from wordcloud import WordCloud
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kilia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2- Constatation des déchets

In [92]:
##Corpus Data1
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(txt_path1, file), 'r', encoding='utf-8') as f:
    before = f.read()

before[:10]


'embre 19.-'

In [72]:
## Corpus Data2
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(txt_path2, file), 'r', encoding='utf-8') as f:
    before = f.read()

before[:10]

"1 ' ' ■ ' "

3- Liste de stop words

In [73]:
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "ville", "que", "voici", "parmi", "chaque", "leurs",
       "lequel", "alors", "laquelle", "trop", "fin", "tant", "ceux", "rien", "grandes",
       "tÃ©l", "dem", "trÃ¨s", "app", "part", "ans","cuis","tÃ©lÃ©ph","sociÃ©tÃ©", "rem","rue", 
       "gar", "grand", "ecrire", "place","monsieur", "quelques", "midi", "brux", "ecrire", "une", "sociÃ©tÃ©",
       "tél", "ecr", "téléph","vitae", "etc", "bur", "réf", "terr","très", "prés", "déjà", "pers",
       "effet", "fem","également", "mén", "vers", "toute", "trav","dès", "porte", "vis", "toujours",
       "démi", "bel", "celui", "jour", "chez", "mod", "samedi", "madame", "liv","tel", "bonne",
       "bon", "mois", "temps", "dimanche","saint", "bonnes", "peu", "près", "villa", "engage", "soir", "libre",
       "cherche", "vendre", "car", "demande", "etat","décès", "partie", "matin", "eau", "reprise","rossel", "premier",
       "première","maison", "jours","avant","cependant", "vue", "but","grande", "adresser", "dame"]
sw = set(sw)

4- Création d'une fonction de nettoyage pour chaque corpus

In [96]:
# Corpus Data1
def clean_text(txt_path1, folder=None):
    if folder is None:
        input_path = f"{txt_path1}.txt"
        output_path = f"{txt_path1}_clean.txt"
    else:
        input_path = f"{folder}/{txt_path1}.txt"
        output_path = f"{folder}/{txt_path1}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [81]:
# Corpus Data2
def clean_text(txt_path2, folder=None):
    if folder is None:
        input_path = f"{txt_path2}.txt"
        output_path = f"{txt_path2}_clean.txt"
    else:
        input_path = f"{folder}/{txt_path2}.txt"
        output_path = f"{folder}/{txt_path2}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

#Appliquer les fonctions à leur corpus respectif

In [97]:
clean_text (txt_path1, folder =None)

'Output has been written in ../data/Data1_Le Drapeau Rouge_clean.txt!'

In [82]:
clean_text (txt_path2, folder =None)

'Output has been written in ../data/Data2_La libre Belgique_clean.txt!'

In [98]:
with open(os.path.join(txt_path1, f'{txt_path1}_clean.txt'), 'r', encoding= 'utf-8') as f:
    after = f.read()

after[:500]

FileNotFoundError: [Errno 2] No such file or directory: '../data/Data1_Le Drapeau Rouge\\../data/Data1_Le Drapeau Rouge_clean.txt'